In [3]:
import pickle
from config import *
import matplotlib.pyplot as plt

In [4]:
comment_tokens = pickle.load(open("comment_tokens.p", "rb"))
all_tokens_set = set()
subreddit_tokens_set = {}
all_tokens_size = {}
train_features = []

In [6]:
for subreddit in subreddits:
    print subreddit
    this_subreddit_tokens = set()
    this_comment_tokens = comment_tokens[subreddit]    
    for comment in this_comment_tokens:
        for t in comment:
            all_tokens_set.add(t)
            this_subreddit_tokens.add(t)
    print len(this_subreddit_tokens)
    subreddit_tokens_set[subreddit] = this_subreddit_tokens

print len(all_tokens_set)

soccer
31984
liverpoolfc
10146
Gunners
9577
reddevils
9517
chelseafc
7868
42319


In [40]:
print comment_tokens["liverpoolfc"][0]

['huge', 'thanks', 'everyone', 'contributed']


In [5]:
train_features = []
test_features = []
train_labels = []
test_labels = []
import math

for subreddit in subreddits:
    this_comment_tokens = comment_tokens[subreddit]
    num_comments = len(this_comment_tokens)
    num_train_comments = int(math.floor(num_comments*0.8))
    num_test_comments =  num_comments - num_train_comments
    print subreddit, num_train_comments, num_test_comments
    train_comments = this_comment_tokens[:num_train_comments]
    test_comments = this_comment_tokens[num_train_comments:]
    print len(train_comments), len(test_comments)
    
    for comment in train_comments:
        #feature = ({word: (word in comment) for word in all_tokens_set}, subreddit)
        feature = ""
        for word in comment:
            feature += word + " "
        train_features.append(feature[:-1])
        train_labels.append(subreddit)
    for comment in test_comments:
        feature = ""
        for word in comment:
            feature += word + " "
        test_features.append(feature[:-1])
        test_labels.append(subreddit)
        
print len(train_features), len(test_features)

soccer 31195 7799
31195 7799
liverpoolfc 5866 1467
5866 1467
Gunners 5120 1280
5120 1280
reddevils 5283 1321
5283 1321
chelseafc 3680 921
3680 921
51144 12788


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = train_features[:]
corpus.extend(test_features)
print len(corpus)
print len(train_features)

vectorizer = CountVectorizer()
vect = vectorizer.fit(corpus)
vocab = vect.vocabulary_

new_vectorizer = CountVectorizer(vocabulary=vocab)
new_vectorizer._validate_vocabulary()
X_train = new_vectorizer.transform(train_features)
X_train.shape
X_test = new_vectorizer.transform(test_features)
X_test.shape

63932
51144


(12788, 42319)

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

clf = MultinomialNB(alpha=.01)
clf.fit(X_train, train_labels)
pred = clf.predict(X_test)
metrics.accuracy_score(test_labels, pred)

0.56670315921176106